In [1]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import time
import glob
from lxml.html import parse
from sphfile import SPHFile
import pydub
import audiosegment
import pandas as pd
from collections import Counter
from bs4 import BeautifulSoup
import sys

/home/junzhez2/.local/lib/python3.5/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [7]:
start_df = pd.read_csv('trainfiles/start.csv')
end_df = pd.read_csv('trainfiles/end.csv')

In [27]:
segments_df = start_df.append(end_df)
speakers = Counter(segments_df['speaker'])
common_speakers = speakers.most_common(20)
segments_by_speakers = {speaker:segments_df[segments_df['speaker']==speaker] for speaker, count in common_speakers}
segments_by_speakers
common_speakers = [common_speaker[0] for common_speaker in common_speakers]

In [41]:
overlay = []
for first_speaker in common_speakers:
    second_speakers = [speaker for speaker in common_speakers if speaker!=first_speaker ]
    first_segments = segments_by_speakers[first_speaker]
    for second_speaker in second_speakers:
        second_segments = segments_by_speakers[second_speaker]
        first_idx = np.arange(290)
        second_idx = np.arange(290)
        np.random.shuffle(first_idx)
        np.random.shuffle(second_idx)
        for i in range(290):
            first_segment = first_segments.iloc[first_idx[i]]
            second_segment = second_segments.iloc[second_idx[i]]
            row = {'first_speaker': first_speaker, 'second_speaker': second_speaker,
                  'first_file': first_segment['segfile'], 'second_file': second_segment['segfile']}
            overlay.append(row)
overlay = pd.DataFrame(overlay)

,first_file,first_speaker,second_file,second_speaker
0,trainfiles/start_segments/ea980127_seg69.npy,peter_jennings,trainfiles/start_segments/g960524__seg43.npy,linden_soles
1,trainfiles/start_segments/ea980119_seg28.npy,peter_jennings,trainfiles/start_segments/g960529__seg44.npy,linden_soles
2,trainfiles/start_segments/ea980126_seg65.npy,peter_jennings,trainfiles/start_segments/g960523__seg6.npy,linden_soles
3,trainfiles/start_segments/ea980109_seg24.npy,peter_jennings,trainfiles/start_segments/eo971227_seg48.npy,linden_soles
4,trainfiles/start_segments/ea980127_seg37.npy,peter_jennings,trainfiles/start_segments/h960517__seg24.npy,linden_soles
5,trainfiles/start_segments/ea980126_seg33.npy,peter_jennings,trainfiles/start_segments/h960517__seg13.npy,linden_soles
6,trainfiles/start_segments/ea980126_seg68.npy,peter_jennings,trainfiles/start_segments/g960521__seg39.npy,linden_soles
7,trainfiles/start_segments/c960610__seg37.npy,peter_jennings,trainfiles/start_segments/h960521__seg123.npy,linden_soles
8,trainfiles/start_segments/ea980126_seg42.npy,peter_jennings,trainfiles/start_segments/eo970830_seg134.npy,linden_soles
9,trainfiles/start_segments/ea980109_seg61.npy,peter_jennings,trainfiles/start_segments/h960517__seg3.npy,linden_soles


In [44]:
overlay_shuffled = overlay.sample(frac=1)
overlay_shuffled[:int(len(overlay)*0.9)].to_csv('overlay-train.csv')
overlay_shuffled[int(len(overlay)*0.9):].to_csv('overlay-test.csv')